## Remote ID Spoofing

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [1]:
import os
import subprocess
from pymavlink import mavutil
import numpy as np
from collections import OrderedDict

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns,global2local,local2global
from helpers.visualization.gazebo_world import update_world
from helpers.math import manhattan_distance
from helpers.navegation_logic import find_path,find_best_waypoint
from helpers.visualization.plots import plot_3d_interactive

## Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/attack_city.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

# Choose Simulator

In [4]:
simulator ='gazebo' # 'QGroundControl' #

## Gonfiguration

In [5]:
wps = [(40, -10, 5),
            (15, -10, 5),
            (0, -10, 5),
            (0, 4, 5),
            (40, 20, 5),
            (15, 20, 5),
            (0, 20, 5)]


use_nav_wps=[True,False]


#GPS_spoofings={'real':[(15, -10, 5),(15, 20, 5)],'fake':[(15, 10, 5),(15, 0, 5)]}
GPS_spoofings={}

wp_plans=[
    [(40, -10,5),(40, 20,5)],
    [(40,  20, 5),(40, -10,5)]
    ]

n_uavs = len(use_nav_wps)


homes=[(*plan[0][:2],0) for plan in wp_plans]
targets=[(*plan[-1][:2],0) for plan in wp_plans]

markers = {'waypoints': {'pos':wps,'color':'yellow'},
           'homes': {'pos':homes,'color':'blue'},
           'targets':{'pos':targets,'color':'green'}}


if GPS_spoofings:
    assert len(GPS_spoofings['real'])==len(GPS_spoofings['fakr']), 'real and fake position must be a one-to-one relation' 
    markers.update({
        'spoofing real':{'pos':GPS_spoofings['real'],'color':'orange'},
        'spoofing fake':{'pos':GPS_spoofings['fake'],'color':'red'}})

# Interactive widget to rotate the plot

plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.4],ground=-0.05)


# Get Local positions

In [6]:
eps=1
headings=n_uavs*[0]
offsets=[(*home,heading) for home,heading in zip(homes,headings)]

In [7]:
homes_np=np.array(homes)
wps_np = np.array(wps)  # Extract keys as a list of tuples


local_wps=global2local(wps_np, homes_np,pairwise=False)
local_plans=[global2local(np.array(plan), home,pairwise=True) for plan,home in zip(wp_plans,homes_np)]


local_GPS_spoofings={key:global2local(np.array(value), homes_np,pairwise=False) for key,value in GPS_spoofings.items()}

In [8]:
if simulator == 'QGroundControl':
    # lat,long,alt,heading
    home_position=(-35.3633245,149.1652241,0,90)
    spawns=find_spawns(home_position, offsets)

## Waypoint seletion algorithm

The path is shown in local coordinates

In [9]:
for i,((_,end),waypoints) in enumerate(zip(local_plans,local_wps)):
        local_path=find_path(np.array((0,0,0)),end, waypoints,eps=1)
        print(f'uav-{i} Best path: {local_path}') 


uav-0 Best path: [[ 0  0  0]
 [ 0  0  5]
 [ 0 30  5]]
uav-1 Best path: [[  0   0   0]
 [  0 -30   5]]


## Launch Simulator

In [10]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

## Create Plan

In [11]:
uavs_arg = []
for i in range(n_uavs):
    uav_arg={'sys_id':i+1,
            'waypoints':local_plans[i],
             'plan': ['check_pos_est','check_prearm','mode_stabilize','mode_guided','arm','takeoff','fly','land'], #
             'wp_margin': 0.5,
             'home':homes[i],
             'wpnv_speed':1}
    uavs_arg.append(uav_arg)

# Launch Vehicle

In [12]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)

In [13]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

## Create UavLogic

In [14]:
uavs=[]
for i,uav_arg in enumerate(uavs_arg):
    uav=VehicleLogic(**uav_arg)
    uavs.append(uav)

WPNAV_SPEED set to 1 m/s (100 cm/s)
vehicle 1 created
WPNAV_SPEED set to 1 m/s (100 cm/s)
vehicle 2 created


## Execute Plan

In [15]:
# import numpy as np
# from scipy.spatial import cKDTree

# # Example positions
# positions = np.array([
#     [0, 0], [1, 1], [2, 2], [5, 5], [16, 16]
# ])

# r = 3.0

# # Build KD-Tree
# tree = cKDTree(positions)

# # Query neighbors (excluding self)
# filtered_neighbors = tree.query_ball_tree(tree, r)
# # Remove self from results
# filtered_neighbors = {i: [j for j in nearby if j != i] for i, nearby in enumerate(filtered_neighbors)}

# # Print results
# for i, nearby in enumerate(filtered_neighbors):
#     print(f"Point {positions[i]} has neighbors: {positions[nearby]}")

# print(filtered_neighbors)

In [16]:
positions_dict = {
    0: [0, 0],
    1: [1, 1],
    2: [2, 2],
    3: [5, 5],
    4: [6, 6]
}

def get_nearest_neighbor(target_id,positions_dict):
    target_position = np.array(positions_dict[target_id])  # Get the position of the target ID
    
    min_distance = float('inf')
    nearest_id = None

    for other_id, position in positions_dict.items():
        if other_id == target_id:
            continue  # Skip itself
        
        distance = np.linalg.norm(target_position - np.array(position))  # Compute Euclidean distance
        
        if distance < min_distance:
            min_distance = distance
            nearest_id = other_id  # Update nearest ID
    
    return nearest_id,min_distance

# Example usage
target_id = 3
nearest_id = get_nearest_neighbor(target_id,positions_dict)
print(f"Nearest neighbor to ID {target_id} is ID {nearest_id}")

Nearest neighbor to ID 3 is ID (4, 1.4142135623730951)


In [17]:
from pymavlink import mavutil


In [18]:
mission_incomplete = n_uavs*[True]
old_dist=n_uavs*[float('inf')]
paths=[[] for _ in range(n_uavs)]
r=5
while any(mission_incomplete):
    #get posiiton
    global_poss={i: local2global(np.array(uav.get_local_position()),np.array(uav.home),pairwise=True) for i,uav in enumerate(uavs) if uav.current_action() in ['takeoff','fly','land']}
    for i,uav in enumerate(uavs):
        # Dynamic Plan
        if uav.current_action()=='fly' and uav.wp_reached:
            # Search algorithm
            final_wp = uav.wps[1]
            curr_wp  = uav.wps[0]

            # GPS spoofing
            if GPS_spoofings:
                spoof_dists=manhattan_distance(curr_wp,local_GPS_spoofings['real'][i])
                spoof_idx=np.argmin(spoof_dists)
                if spoof_dists[spoof_idx]<1:
                    curr_wp=local_GPS_spoofings['fake'][i][spoof_idx]

            # Search algorithm
            curr_wp= find_best_waypoint(curr_wp, final_wp, local_wps[i],eps=eps)
            
            # Modify wp_plan
            if np.array_equal(curr_wp, final_wp):
                path=np.stack([final_wp],axis=0)
            else:
                path=np.stack([curr_wp,final_wp],axis=0)

            uav.update_waypoints(path)

            paths[i].append(curr_wp)

        #This is temporal
        if i in global_poss.keys():
            _,min_distance=get_nearest_neighbor(i,global_poss)
            if min_distance < r and min_distance<old_dist[i]:
                print(uav.current_action())
                #uav.stop_position_control()
                uav.move_left(speed=1)
                print('moved left')
            else:
                print(f'uav {uav.sys} act plan')
                mission_incomplete[i] = uav.act_plan()
            old_dist[i]=min_distance
        else:
            mission_incomplete[i] = uav.act_plan()
for path in paths:
    print(path)
        

vehicle 1: action check_pos_est is done
vehicle 2: action check_pos_est is done
vehicle 1: action check_prearm is done
vehicle 2: action check_prearm is done
ack?
ack end
vehicle 1: action mode_stabilize is done
ack?
ack end
vehicle 2: action mode_stabilize is done
ack?
ack end
vehicle 1: action mode_guided is done
ack?
ack end
vehicle 2: action mode_guided is done
ack?
ack end
vehicle 1: action arm is done
ack?
ack end
vehicle 2: action arm is done
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 act plan
uav 2 act plan
uav 1 

KeyboardInterrupt: 

## Kill all related process

In [ ]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")